<a href="https://colab.research.google.com/github/ethansong206/Climate-Plus-Project/blob/main/DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import necessary packages for the code

In [15]:
import pandas as pd
import numpy as np
from datetime import datetime
import re

Load the Duke Dining csv file and make the data easier to handle. This includes but is not limited to:

1) Renaming locations

2) Changing the date from a string to a date object



In [3]:
DiningDataFull = pd.read_csv('Climate+ Data 2019 thru 2023.csv', 
                             dtype = {'Priority 1': str, 'Priority 2': str,
                                      'Priority 3': str})
#print(DiningDataFull['Unit Name'].value_counts()) # Show how many entries there are of each location

#below code to make 'Unit Name' column easier to handle
#can add more lines given more locations
def location_rename(location):
    if("Marketplace" in location): #Combine data for Marketplace Kitchen and Marketplace Special Event
        return "Marketplace"
    if("Marine Lab" in location):
        return "DuML"
    if("Trinity" in location):
        return "Trinity"
    if("Freeman" in location):
        return "Freeman"
    return None

DiningDataFull['Unit Name'] = DiningDataFull.apply(lambda d: location_rename(d['Unit Name']), axis = 1)

DiningDataFull['Purchase Date'] = DiningDataFull.apply(lambda d: datetime.strptime(d['Purchase Date'], '%m/%d/%Y').date(), axis = 1)

print("Rows in DiningDataFull: ", DiningDataFull.shape[0])

Rows in DiningDataFull:  90534


#**\*\*\*WIP\*\*\***

Extract the unit names for each item into a new column called `Unit`, then rename each unit into a more simple label (i.e. LB CS to LB). Convert cans and bottle cases into the equivalent value in OZ. Make a new column called `Total Food Amount` for the total amount of food in the current unit of measurement, before converting to grams. Make a new column called `Unit(g)` for converting all units to the equivalent value in grams, then filter out main descriptor word(s) in `Vendor Item Description` into a new column called `Food Name`.

---
Note on Exclusion: Some units are left out in this first calculation of emissions as they are either not directly food (i.e. gloves) or are too difficult to go through individually and find a measurement that is not ambiguous. The total number of entries left out is 12171. **The number of non-food items in this amount can be calculated later.**

---
Note on Conversions: The column `Vendor Item Purchase Unit` is in the format x/y n which should be read as x bags of y units of n food.

---

Take the column with units in grams and multiply by the `Receive Quantity` if provided into a new column called `Total Grams`. If there is no value in `Receive Quantity`, then assume it is the value 1.


In [29]:
#this code extracts just the unit information
DiningDataFull['Unit'] = DiningDataFull['Vendor Item Purchase Unit'].str.extract(r" ?([A-Za-zÀ-ÿ ]*)$").astype(str)
#below code to simplify redundant labels (i.e. LB CS to LB)
def unit_rename(unit):
    if('LB' in unit) | ('lb' in unit) | ('Lb' in unit) | ('Pound' in unit):
        return 'LB'
    if('OZ' in unit) | ('oz' in unit) | ('Oz' in unit) | ('Z' in unit):
        return 'OZ'
    if('GA' in unit) | ('Gal' in unit):
        return 'GA'
    if('QT' in unit):
        return 'QT'
    if('PT' in unit) | ('Pint' in unit) | ('PINT' in unit):
        return 'PT'
    if('LT' in unit):
        return 'LT' 
    if('BU' in unit) | ('Bushel' in unit): #bushels
        return 'BU'
    if('KG' in unit):
        return 'KG' 
    if('GR' in unit): #grams
        return 'GR' 
    if('ML' in unit) | ('ml' in unit) | ('Ml' in unit): #milliliters
        return 'ML'
    if('CN' in unit) | ('Can' in unit):
        return 'CN'
    if('Bottle Case' in unit):
        return 'Bottle Case'
    return None
#Notes: Bottle Case is 64 oz each, find # of can and translate to oz, anything with EA is not included for now (~9000)
DiningDataFull['Unit'] = DiningDataFull.apply(lambda d: unit_rename(d['Unit']), axis = 1)

#make a new dataset for just entries with known units
DiningDataReduced = DiningDataFull[DiningDataFull['Unit'].notna()]
#print(DiningDataReduced.head())
print("Excluded rows: ", DiningDataFull.shape[0] - DiningDataReduced.shape[0])

#CONVERT UNITS
#first convert cans to OZ
#using estimates for weight of cans through https://food.unl.edu/article/how-interpret-can-size-numbers
DiningDataReduced = DiningDataReduced.replace({'#10 CN' : '110.5 OZ'}, regex = True)
DiningDataReduced = DiningDataReduced.replace({'#300 CN' : '15 OZ'}, regex = True)
DiningDataReduced = DiningDataReduced.replace({'CN' : 'OZ'})
#next convert bottle case to OZ
DiningDataReduced = DiningDataReduced.replace({' Bottle Case' : '/64 OZ'}, regex = True)
DiningDataReduced = DiningDataReduced.replace({'Bottle Case' : 'OZ'})

#find total amount of food before converting to grams
#DO THIS NEXT
DiningDataReduced['Total Food Amount'] = DiningDataReduced['Vendor Item Purchase Unit'] \
                                              .str.extract(r'^[a-zA-Z]* ?([0-9]*/?[0-9.]*-?[0-9.]*)')

print(DiningDataReduced[{'Vendor Item Purchase Unit', 'Total Food Amount'}][0:50])
print(DiningDataReduced.head())

Excluded rows:  11638
   Total Food Amount Vendor Item Purchase Unit
0                6/5                    6/5 LB
1                6/5                    6/5 LB
2              6/4.5                  6/4.5 LB
3               2/10                   2/10 LB
4                6/5                    6/5 LB
5               2/10                   2/10 LB
6                6/5                    6/5 LB
7                6/5                    6/5 LB
8                2/5                    2/5 LB
9               2/10                   2/10 LB
10               6/4                    6/4 LB
11               3/5                    3/5 LB
12             4/2.5                  4/2.5 LB
13              4/18                 4/18/3 OZ
14              2/10                   2/10 LB
15               6/5                    6/5 LB
16              2/32              2/32/2.75 OZ
17               2/5                    2/5 LB
18               6/5                    6/5 LB
19               4/4                  

<ipython-input-29-02b0eb250f4b>:53: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  print(DiningDataReduced[{'Vendor Item Purchase Unit', 'Total Food Amount'}][0:50])


Making one dataset for each location and each year

In [4]:
location_col = DiningDataReduced['Unit Name']
DiningData_Marketplace = DiningDataReduced[location_col == "Marketplace"]
DiningData_DuML = DiningDataReduced[location_col == "DuML"]
DiningData_Trinity = DiningDataReduced[location_col == "Trinity"]
DiningData_Freeman = DiningDataReduced[location_col == "Freeman"]

date_col = DiningDataReduced['Purchase Date']
DiningData_2019 = DiningDataReduced[(date_col >= pd.Timestamp(2019, 1, 1)) & (date_col < pd.Timestamp(2020, 1, 1))]
DiningData_2020 = DiningDataReduced[(date_col >= pd.Timestamp(2020, 1, 1)) & (date_col < pd.Timestamp(2021, 1, 1))]
DiningData_2021 = DiningDataReduced[(date_col >= pd.Timestamp(2021, 1, 1)) & (date_col < pd.Timestamp(2022, 1, 1))]
DiningData_2022 = DiningDataReduced[(date_col >= pd.Timestamp(2022, 1, 1)) & (date_col < pd.Timestamp(2023, 1, 1))]
DiningData_2023 = DiningDataReduced[(date_col >= pd.Timestamp(2023, 1, 1)) & (date_col < pd.Timestamp(2024, 1, 1))]

<ipython-input-4-51466aaa99e5>:8: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  DiningData_2019 = DiningDataReduced[(date_col >= pd.Timestamp(2019, 1, 1)) & (date_col < pd.Timestamp(2020, 1, 1))]
<ipython-input-4-51466aaa99e5>:9: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  DiningData_2020 = DiningDataReduced[(date_col >= pd.Timestamp(2020, 1, 1)) & (date_col < pd.Timestamp(2021, 1, 1))]
<ipython-input-4-51466aaa99e5>:10: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-c